In [1]:
import keras
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
import time

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector

import datetime

Using TensorFlow backend.


In [2]:
HIDDEN_SIZE = 1024
MAX_NODES = 100
MAX_VC_DIM = 10


In [3]:
# create and fit the LSTM network
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAX_NODES, MAX_VC_DIM)))
model.add(RepeatVector(MAX_NODES))
model.add(LSTM(HIDDEN_SIZE, return_sequences=True))
model.add(Dense(MAX_VC_DIM))

model.compile(loss='mean_squared_error',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1024)              4239360   
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 100, 1024)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 1024)         8392704   
_________________________________________________________________
dense_1 (Dense)              (None, 100, 10)           10250     
Total params: 12,642,314
Trainable params: 12,642,314
Non-trainable params: 0
_________________________________________________________________


In [4]:
data = np.load('../../../expt/datasets/2019-05-28 11:37:37.368580_30.npy')

In [5]:
X, y = data[0], data[1]

In [7]:
X.shape

(30, 100, 10)

In [8]:
model.fit(X, y, epochs=10, validation_split=0.33)

Instructions for updating:
Use tf.cast instead.
Train on 20 samples, validate on 10 samples
Epoch 1/10
20/20 [==============================] - 8s 387ms/step - loss: 0.0116 - acc: 0.0475 - val_loss: 0.0106 - val_acc: 0.6770
Epoch 2/10
20/20 [==============================] - 6s 281ms/step - loss: 0.0119 - acc: 0.6805 - val_loss: 0.0105 - val_acc: 0.5980
Epoch 3/10
20/20 [==============================] - 6s 281ms/step - loss: 0.0117 - acc: 0.6605 - val_loss: 0.0105 - val_acc: 0.0000e+00
Epoch 4/10
20/20 [==============================] - 6s 283ms/step - loss: 0.0117 - acc: 0.0000e+00 - val_loss: 0.0105 - val_acc: 0.0000e+00
Epoch 5/10
20/20 [==============================] - 6s 292ms/step - loss: 0.0117 - acc: 0.0165 - val_loss: 0.0105 - val_acc: 0.0000e+00
Epoch 6/10
20/20 [==============================] - 6s 288ms/step - loss: 0.0116 - acc: 0.0200 - val_loss: 0.0105 - val_acc: 0.6640
Epoch 7/10
20/20 [==============================] - 6s 285ms/step - loss: 0.0116 - acc: 0.6635 - val

In [9]:
!pwd

/home/keshava/Desktop/TPM_from_VCS/expt/TPM_from_VCS/models


In [25]:
model.save(str(datetime.datetime.now()) + '.h5')